In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Función para cargar y preprocesar datos
def cargar_datos():
    # Cargar datos desde Excel
    datos = pd.read_excel("datos_clean.xlsx", sheet_name="Hoja4")
    
    # Convertir variables categóricas en tipo categoría
    for col in datos.select_dtypes(include=['object']).columns:
        datos[col] = datos[col].astype('category')
    
    # Tratar valores faltantes en numéricos con la mediana
    for col in datos.select_dtypes(include=['number']).columns:
        datos[col].fillna(datos[col].median(), inplace=True)
    
    # Tratar valores faltantes en categóricos con "Desconocido"
    for col in datos.select_dtypes(include=['category']).columns:
        datos[col] = datos[col].cat.add_categories(["Desconocido"]).fillna("Desconocido")
    
    # Convertir variables categóricas en dummies (One-Hot Encoding)
    datos = pd.get_dummies(datos, columns=datos.select_dtypes(include=['category']).columns, drop_first=False)
    
    return datos

# Cargar y dividir los datos
datos = cargar_datos()
print(f"Tamaño del conjunto de datos original: {datos.shape}")
print("Primeras filas del dataset original:")
print(datos.head())

# Separar en datos de entrenamiento y prueba
X = datos.drop(columns=["price"])
y = datos["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Optimización de hiperparámetros en Random Forest
rf = RandomForestRegressor(n_estimators=3000, max_features='sqrt', min_samples_leaf=2, max_leaf_nodes=500, random_state=123)
rf.fit(X_train, y_train)

# Hacer predicciones
y_pred = rf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Mostrar métricas optimizadas
print(f"\nRMSE Optimizado (Random Forest): {rmse}")
print(f"R² Optimizado (Random Forest): {r2}")

# Función para predecir precio basado en input del usuario
def predecir_precio(modelo, trainData):
    print("\nIngrese los datos para la predicción:")
    
    # Solicitar entrada del usuario
    bedrooms = float(input("Número de habitaciones: "))
    bathrooms = float(input("Número de baños: "))
    square_feet = float(input("Tamaño en pies cuadrados: "))
    
    # Obtener nombres de las variables dummy del estado
    columnas_estado = [col for col in trainData.columns if col.startswith("state_")]
    
    # Extraer nombres reales de estados, excluyendo "Desconocido"
    estados_disponibles = [col.replace("state_", "") for col in columnas_estado if col != "state_Desconocido"]
    
    if not estados_disponibles:
        print("⚠️ No se encontraron estados disponibles en los datos. Verifique el preprocesamiento.")
        return
    
    # Mostrar opciones de estado al usuario
    print("\nEstados disponibles:")
    for i, estado in enumerate(estados_disponibles, 1):
        print(f"{i}: {estado}")
    
    # Pedir al usuario que elija un estado
    seleccion = int(input("\nSeleccione el número del estado: "))
    
    # Validar la selección
    if seleccion < 1 or seleccion > len(estados_disponibles):
        print("⚠️ Error: Selección inválida. Intente de nuevo ingresando un número de la lista.")
        return
    
    # Obtener el estado seleccionado
    estado_seleccionado = estados_disponibles[seleccion - 1]
    
    # Crear un DataFrame con los datos ingresados por el usuario
    datos_usuario = pd.DataFrame({
        "bedrooms": [bedrooms],
        "bathrooms": [bathrooms],
        "square_feet": [square_feet]
    })
    
    # Agregar todas las variables dummy del estado con valor 0
    for col in columnas_estado:
        datos_usuario[col] = 0
    
    # Activar la variable correspondiente al estado elegido (poner 1)
    estado_dummy = f"state_{estado_seleccionado}"
    if estado_dummy in datos_usuario.columns:
        datos_usuario[estado_dummy] = 1
    
    # Asegurar que las columnas coincidan con las del modelo
    datos_usuario = datos_usuario.reindex(columns=X_train.columns, fill_value=0)
    
    # Hacer la predicción
    precio_predicho = modelo.predict(datos_usuario)
    
    # Mostrar resultado
    print(f"\n📢 Precio estimado: {round(precio_predicho[0], 2)}")



C:\Users\Usuario\AppData\Local\Temp\ipykernel_20964\1450193810.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos[col].fillna(datos[col].median(), inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20964\1450193810.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

Tamaño del conjunto de datos original: (9470, 56)
Primeras filas del dataset original:
   price  bathrooms  bedrooms  square_feet  state_AK  state_AL  state_AR  \
0    625        1.0         1          200     False     False     False   
1    600        1.0         1          200     False     False     False   
2    544        1.0         1          200     False     False     False   
3    450        1.0         1          200     False     False     False   
4    750        1.0         1          215     False     False     False   

   state_AZ  state_CA  state_CO  ...  state_TN  state_TX  state_UT  state_VA  \
0     False     False     False  ...     False     False     False     False   
1     False     False     False  ...     False     False     False     False   
2     False     False     False  ...     False     False     False     False   
3      True     False     False  ...     False     False     False     False   
4     False     False     False  ...     False     False

In [9]:
predecir_precio(rf, X_train)


Ingrese los datos para la predicción:

Estados disponibles:
1: AK
2: AL
3: AR
4: AZ
5: CA
6: CO
7: CT
8: DC
9: DE
10: FL
11: GA
12: HI
13: IA
14: ID
15: IL
16: IN
17: KS
18: KY
19: LA
20: MA
21: MD
22: ME
23: MI
24: MN
25: MO
26: MS
27: MT
28: NC
29: ND
30: NE
31: NH
32: NJ
33: NM
34: NV
35: NY
36: OH
37: OK
38: OR
39: PA
40: RI
41: SC
42: SD
43: TN
44: TX
45: UT
46: VA
47: VT
48: WA
49: WI
50: WV
51: WY

📢 Precio estimado: 1747.28
